In [1]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_feather('data/nasdaq_etfs.feather')
df # contains daily percentage returns (NOTE: 0.747 means 0.747% not 74.7%)

Ticker,AAXJ,ACWI,ACWX,AFK,AGQ,AIA,AIVI,AIVL,AOA,AOK,...,XRT,XSD,XSMO,XSVM,YANG,YCL,YCS,YINN,YXI,ZSL
Date,,,,,,,,,,,,,,,,,,,,,
2010-07-13,0.747621,1.562114,1.686754,1.853138,2.700842,0.854489,1.788925,1.268345,1.884208,0.435343,...,2.734448,2.155864,3.351985,3.351523,-2.632439,0.140403,-0.369203,2.300923,-0.936347,-2.774568
2010-07-14,-0.380087,0.372102,0.263322,0.034330,1.457285,0.051332,0.051002,-0.168632,-0.470754,0.000000,...,-0.452477,0.065970,0.154452,-0.603311,2.571722,0.385555,-0.476437,-2.663477,1.155257,-1.634958
2010-07-15,-0.763130,0.098904,0.157541,0.274559,-0.511807,-0.821128,1.603865,0.192969,-0.135120,0.108410,...,0.588216,-0.131862,-0.462618,0.379324,2.828656,1.815644,-1.968086,-2.827608,0.000000,0.302206
2010-07-16,-2.453279,-2.765485,-2.884077,-1.540060,-4.829074,-2.664938,-2.706147,-2.167090,-2.063619,-0.721728,...,-3.561905,-2.947607,-3.718039,-3.930446,5.470490,2.023320,-1.627784,-5.882389,3.612906,5.122025
2010-07-19,1.144909,0.634864,0.539948,-0.451860,-2.580644,1.302489,0.566597,1.058404,0.207255,0.072730,...,0.385863,2.606528,0.402229,0.393409,-2.874933,-0.336136,0.386099,3.158260,-0.420834,2.694180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-07,-1.583303,-0.870629,-1.126528,-0.145141,-0.525887,-1.604603,-1.320403,-0.700031,-0.815446,-0.564830,...,-0.855754,-2.682102,-1.628708,-1.290079,-0.125316,-1.353709,1.789122,0.101551,0.274723,0.564969
2025-07-08,0.682508,0.170994,0.561427,-0.678292,-0.589672,0.722024,0.543588,-0.017628,0.217628,0.000000,...,-0.175131,2.151325,0.146518,0.392082,-3.011290,-0.531212,0.820254,2.865841,-1.187216,0.481544
2025-07-09,-0.145264,0.543139,0.459768,-0.243899,-1.513598,-0.494378,0.873359,0.070511,0.494636,0.413116,...,0.488721,1.100442,1.111927,0.353354,4.075033,0.133514,-0.418411,-4.142012,0.554533,1.597443
